In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [3]:
pib = pd.read_csv(
    "https://raw.githubusercontent.com/alura-cursos/dataviz-graficos-composicao-relacionamento/main/dados/pib_br_2002_2020_estados.csv"
)
pib.head()

,ano,sigla_uf,regiao,pib,impostos_liquidos,va,va_agropecuaria,va_industria,va_servicos,va_adespss
0,2002,RO,Norte,7467629886,839731192,6627898698,715526872,1191090432,2484579193,2236702207
1,2003,RO,Norte,9425010486,1108434936,8316575548,1278658831,1216605061,3376727040,2444584625
2,2004,RO,Norte,11004641436,1288806654,9715834778,1288515348,1674933817,3986529419,2765856199
3,2005,RO,Norte,12511821181,1476144194,11035676990,1342222120,1887932121,4603783904,3201738843
4,2006,RO,Norte,13054713344,1613809974,11440903374,1238006193,2210692147,4320526746,3671678293


In [4]:
pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ano                513 non-null    int64 
 1   sigla_uf           513 non-null    object
 2   regiao             513 non-null    object
 3   pib                513 non-null    int64 
 4   impostos_liquidos  513 non-null    int64 
 5   va                 513 non-null    int64 
 6   va_agropecuaria    513 non-null    int64 
 7   va_industria       513 non-null    int64 
 8   va_servicos        513 non-null    int64 
 9   va_adespss         513 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 40.2+ KB


# Como estão dispostos os valores agregados de serviços por região do Brasil no ano de 2018?

In [5]:
df_services_2018 = (
    pib.query("ano == 2018").groupby("regiao")["va_servicos"].sum().reset_index()
)
df_services_2018

,regiao,va_servicos
0,Centro-Oeste,296250896281
1,Nordeste,433449662991
2,Norte,133394247965
3,Sudeste,1933221862566
4,Sul,546627330178


In [6]:
fig = px.pie(
    df_services_2018,
    values="va_servicos",
    names="regiao",
    color="regiao",
    color_discrete_sequence=px.colors.qualitative.Dark24,
    height=600,
    width=800,
)
fig.update_traces(
    textposition="inside",
    textinfo="percent+label",
    textfont=dict(color="white"),
    texttemplate="<b>%{label}<br>%{percent}",
    hoverinfo="skip",
    hovertemplate=None,
)
fig.update_layout(
    title_text="<b>Composição dos valores agregados do setor de serviços em 2018",
    showlegend=False,
)
fig.show()

# Qual a participação da Região Norte nos valores agregados da indústria de todo Brasil nos anos de 2010 e 2020?

In [7]:
pib_norte_2010 = pib.query("ano == 2010")[["regiao", "pib"]]
pib_norte_2010.loc[pib_norte_2010["regiao"] != "Norte", "regiao"] = "Outros"
pib_norte_2010 = pib_norte_2010.groupby("regiao").sum()
pib_norte_2010

,pib
regiao,
Norte,207093644805
Outros,3678753355168


In [8]:
pib_norte_2020 = pib.query("ano == 2020")[["regiao", "pib"]]
pib_norte_2020.loc[pib_norte_2020["regiao"] != "Norte", "regiao"] = "Outros"
pib_norte_2020 = pib_norte_2020.groupby("regiao").sum()
pib_norte_2020

,pib
regiao,
Norte,478173048893
Outros,7131423951126


In [9]:
# Defina as cores para o gráfico
colors = ["#1f77b4", "#7f7f7f"]

# Crie os subplots com o tipo "pie"
fig = sp.make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

# Crie o primeiro gráfico de pizza
fig.add_trace(
    go.Pie(
        labels=pib_norte_2010.index,
        values=pib_norte_2010["pib"],
        textinfo="label+percent",
        marker=dict(colors=colors),
        hole=0.6,
        sort=False,
        rotation=270,
        hoverinfo="skip",
        hovertemplate=None,
    ),
    row=1,
    col=1,
)

# Crie o segundo gráfico de pizza
fig.add_trace(
    go.Pie(
        labels=pib_norte_2020.index,
        values=pib_norte_2020["pib"],
        textinfo="label+percent",
        marker=dict(colors=colors),
        hole=0.6,
        sort=False,
        rotation=270,
        hoverinfo="skip",
        hovertemplate=None,
    ),
    row=1,
    col=2,
)

# Personalize o layout
fig.update_layout(
    title="<b>Participação da Região Norte nos Valores Agregados da Indústria</b><br>Em 2010 e 2020",
    title_font=dict(size=18),
    showlegend=False,
    height=600,
    width=800,
    annotations=[
        dict(
            text="2010",
            x=0.18,
            y=0.5,
            font_size=20,
            showarrow=False,
        ),
        dict(
            text="2020",
            x=0.82,
            y=0.5,
            font_size=20,
            showarrow=False,
        ),
    ],
)

# Mostre o gráfico
fig.show()

A região Norte registrou um incremento de 0,95% no período de 2010 a 2020 no Produto Interno Bruto (PIB) proveniente do setor industrial quando comparado com as outras regiões do Brasil

# Como está composto, em valores absolutos, o PIB por região dos 4 últimos anos do conjunto de dados (2017 a 2020)?

In [10]:
anos = [x for x in range(2017,2021)]
pib_regiao_17_20 = pib.query("ano == @anos")[["ano","regiao","pib"]]
pib_regiao_17_20["pib"] = (pib_regiao_17_20["pib"] / 1e12).round(2)
pib_regiao_17_20 = pd.crosstab(index = pib_regiao_17_20.ano, columns = pib_regiao_17_20.regiao,
                            values = pib_regiao_17_20.pib, aggfunc = "sum")
pib_regiao_17_20 = pib_regiao_17_20[["Sudeste", "Sul", "Nordeste", "Centro-Oeste", "Norte"]].reset_index()
pib_regiao_17_20

regiao,ano,Sudeste,Sul,Nordeste,Centro-Oeste,Norte
0,2017,3.48,1.12,0.95,0.66,0.36
1,2018,3.72,1.20,1.01,0.70,0.39
2,2019,3.92,1.27,1.04,0.73,0.43
3,2020,3.95,1.31,1.09,0.79,0.49


In [11]:
# Defina as cores para o gráfico
cores = px.colors.qualitative.G10

# Crie o gráfico de barras empilhadas
fig = go.Figure()

for i, regiao in enumerate(["Sudeste", "Sul", "Nordeste", "Centro-Oeste", "Norte"]):
    fig.add_trace(go.Bar(
        x=pib_regiao_17_20["ano"],
        y=pib_regiao_17_20[regiao],
        name=regiao,
        marker_color=cores[i],
        text=pib_regiao_17_20[regiao],
        textposition="inside",
        texttemplate="<b>%{text:.2f} Tri"
    ))

fig.update_layout(
    title="<b>Composição do PIB por região</b><br>De 2017 a 2020 (em trilhões de reais)",
    title_font=dict(size=18),
    xaxis_title="",
    yaxis_title="",
    legend=dict(
        title="Região",
        title_font=dict(size=12),
        font=dict(size=10),
    ),
    showlegend=True,
    barmode='stack',
    height=600,
    width=800,
    template="plotly_white"
)
fig.update_yaxes(showticklabels=False, showgrid=False)
fig.update_xaxes(tickvals=[2017, 2018, 2019, 2020], ticktext=["2017", "2018", "2019", "2020"], tickfont=dict(size=14))

fig.update_traces(
    hovertemplate="<b>%{x}<br>R$ %{text} tri",
)

fig.show()


# Como está composto os valores agregados brutos dos bens e serviços (em valor absoluto) em relação aos Estados da região Norte no ano de 2020?

In [12]:
pib_norte_2020 = (pib
                  .query("ano == 2020 and regiao == 'Norte'")[["sigla_uf","va_servicos", "va_industria", "va_adespss", "va_agropecuaria"]]
                  .sort_values(by="va_servicos", axis=0)
                  )
pib_norte_2020

,sigla_uf,va_servicos,va_industria,va_adespss,va_agropecuaria
75,RR,5278754961,1706511471,6538065265,1000907462
113,AP,6481918486,2134852286,8255441052,339906135
37,AC,6590543336,1191345379,6031050521,983531817
132,TO,15659235395,4408431382,11408069014,8033173625
18,RO,19060688172,8285675423,12000339417,6891411669
56,AM,34795836262,35839810630,20210620577,5114449144
94,PA,56395092425,84173852308,37614037902,19730656823


In [45]:
colors = ['color:' + color for color in px.colors.qualitative.G10]
colors

['color:#3366CC',
 'color:#DC3912',
 'color:#FF9900',
 'color:#109618',
 'color:#990099',
 'color:#0099C6',
 'color:#DD4477',
 'color:#66AA00',
 'color:#B82E2E',
 'color:#316395']

In [61]:
# Define the colors for the plot
cores = px.colors.qualitative.G10
nomes = ["Serviços", "Indústria", "ADESPSS", "Agropecuária"]

# Create the stacked bar plot
fig = go.Figure()

for i, col in enumerate(["va_servicos", "va_industria", "va_adespss", "va_agropecuaria"]):
    fig.add_trace(go.Bar(
        y=pib_norte_2020["sigla_uf"],
        x=pib_norte_2020[col],
        name=nomes[i],
        orientation='h',
        marker_color=cores[i],
        text=(pib_norte_2020[col] / 1e9).apply(lambda x: f"{x:.0f} Bi".replace(".",",")),
        textposition="inside",
        texttemplate="<b>%{text}",
        hovertemplate="<b>%{y}<br>R$ %{text}"
    ))

fig.update_layout(
    title="<b>Composição dos Valores Agregados Brutos dos Bens e Serviços na Região Norte do Brasil - 2020</b><br>Valor Agregado (em bilhões de reais)",
    title_font=dict(size=18),
    xaxis_title="",
    yaxis_title="Estados",
    legend=dict(
        title="Setor",
        title_font=dict(size=12),
        font=dict(size=10),
    ),
    showlegend=False,
    barmode='stack',
    template="plotly_white",
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_yaxes(tickfont=dict(size=12))

text_annotations = '| '
for i in range(len(nomes)):
    text_annotations = text_annotations + '<b><span style="color:' + cores[i] + '">' + nomes[i] + '</span></b> | ' 

fig.update_traces(customdata=['color:' + color for color in px.colors.qualitative.G10])
fig.add_annotation(
    x=28e9,
    y=7,
    text=text_annotations,
    showarrow=False
)

fig.show()